In [1]:
import openai
import pandas as pd
from dotenv import load_dotenv
import sys
import os
from openai import OpenAI
import re
import concurrent.futures

In [ ]:
def clean_acs_data(df):
    """
    Cleans and processes ACS (American Community Survey) data by applying 
    filtering criteria and calculating derived metrics.

    Steps:
    1. Removes rows where 'median_household_income' is not greater than 0.
    2. Removes rows where 'median_age' is not greater than 0.
    3. Filters out locations with a population of 1,000 or fewer.
    4. Drops rows with missing values in 'num_intersections'.
    5. Computes 'population_density' as population divided by area in square kilometers.
    6. Computes 'pct_bachelor' as the percentage of people with a bachelor's degree 
       among those 25 years or older.
    7. Selects and retains only relevant columns.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing ACS data.

    Returns:
    pd.DataFrame: A cleaned and processed DataFrame with selected columns and computed metrics.
    """

    df_final = df.loc[df['median_household_income'] > 0]
    df_final = df_final.loc[df_final['median_age'] > 0]
    df_final = df_final.loc[df_final['population'] > 1_000]
    df_final = df_final.dropna(subset='num_intersections')
    df_final['population_density'] = df_final['population'] / df_final['area_sqkm']
    df_final['pct_bachelor'] = df_final['bachelor_degree'] / df_final['total_25_plus_education']
    keep_cols = [
        'state_name', 'town', 'median_household_income', 
        'population', 'median_age', 'intersection_density', 
        'population_density', 'pct_bachelor']
    df_final = df_final[keep_cols]
    return df_final



In [5]:
def get_county_with_openai(client, town, state):
    """
    Returns only the county name and two-letter state abbreviation for a given town and state.
    The response is expected to be exactly in the format "<County>, <State>".
    If extra text is returned, the function attempts to extract the desired format using regex.
    """
    prompt = (
        f"Return only the county name and the two-letter state abbreviation for the location "
        f"'{town}, {state}, USA' in the exact format '<County>, <State>'. "
        "Do not include any additional text, explanation, or apologies."
    )
    
    response = client.chat.completions.create(
        model="gpt-4",  # or use 'gpt-4' if you have access
        messages=[
            {"role": "system", "content": "You are a data extraction assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    
    county_result = response.choices[0].message.content.strip()
    
    # Use regex to enforce the format: text, comma, space, two uppercase letters.
    match = re.search(r'^([^,]+),\s*([A-Z]{2})$', county_result)
    if match:
        return match.group(0)
    else:
        # If the exact pattern wasn't matched, try to extract the first occurrence.
        match = re.search(r'([A-Za-z\s]+),\s*([A-Z]{2})', county_result)
        if match:
            return f"{match.group(1).strip()}, {match.group(2).strip()}"
        else:
            # Fallback: return None.
            return None

In [6]:
def add_county_column(df, town_col='town', state_col='state_name'):
    """
    For each row in the DataFrame, constructs a location string (e.g., "Annapolis, MD, USA"),
    uses the OpenAI API to retrieve the county in the exact format "<County>, <State>",
    and populates a new column 'county' with the result.
    """
    df = df.copy()
    df['county'] = df.apply(
        lambda row: get_county_with_openai(row[town_col], row[state_col]),
        axis=1
    )
    return df


def add_county_column_parallel(df, town_col='town', state_col='state_name', max_workers=5):
    """
    Processes the DataFrame in parallel using a ThreadPoolExecutor.
    For each row, it calls get_county_with_openai to obtain the county,
    and then populates a new 'county' column.
    
    Parameters:
      - df: The input DataFrame.
      - town_col: Name of the column containing the town.
      - state_col: Name of the column containing the state abbreviation.
      - max_workers: Maximum number of concurrent threads.
    
    Returns:
      - A copy of df with a new column 'county'.
    """
    df = df.copy()
    # Create a function that processes one row.
    def process_row(row):
        return get_county_with_openai(row[town_col], row[state_col])
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Use executor.map to run process_row for each row (iterrows returns each row as a Series)
        results = list(executor.map(process_row, (row for _, row in df.iterrows())))
    
    df['county'] = results
    return df


In [ ]:
def main():
    """
    Main function to load ACS data, process it by adding county information, 
    and save the processed DataFrame.

    Steps:
    1. Load environment variables (API keys).
    2. Initialize the OpenAI client.
    3. Add the current directory to the system path.
    4. Define input and output file paths.
    5. Load the ACS dataset.
    6. Add county information using `add_county_column()`.
    7. Save the processed DataFrame to a CSV file.

    Returns:
    None
    """

    # Load environment variables
    print("Loading environment variables...")
    load_dotenv()

    # Initialize OpenAI client
    print("Initializing OpenAI client...")
    client = OpenAI()
    openai.api_key = os.getenv("OPENAI_API_KEY")

    # Ensure the script can access necessary modules
    sys.path.append(os.path.abspath('.'))
    print("Added current directory to system path.")

    # Define file paths
    input_path = os.path.join('..', 'data', 'acs_with_area_and_intersection.csv')
    output_path = os.path.join('..', 'data', 'acs_with_county.csv')
    print(f"Input file path: {input_path}")
    print(f"Output file path: {output_path}")

    # Load data
    print("Loading ACS data...")
    df_acs = pd.read_csv(input_path).pipe(clean_acs_data)
    print("Data loaded successfully.")

    # Process data by adding county information
    print("Adding county column...")
    df_acs_with_county = add_county_column(df_acs)
    print("County column added successfully.")

    # Save the processed DataFrame
    print(f"Saving processed data to {output_path}...")
    df_acs_with_county.to_csv(output_path, index=False)
    print("File saved successfully.")



In [ ]:
main()